# Getting started

This page contains several examples:

* A minimal working example
* An example using dynamic exercises
* An example with several days in a week
* And finally, a realistic program

The examples below build on each other and become progressively more complex, and with the additional complexity comes additional flexibility and power. After reading this page, you will be able to efficiently create your own training programs.

## Minimal working example

In [1]:
from streprogen import StaticExercise, Day, Program

# Create a  3 week training program
program = Program("Minimal program", duration=4)

# Create a static exercise to a day
squats = StaticExercise("Squats", "5 x 5 @ 80kg")
day = Day(exercises=[squats])

# Add the day to the program and render it
program.add_days(day)
program.render()
print(program)

----------------------------------------------------------------
Program: Minimal program

Program parameters
  duration: 4
  reps_per_exercise: 25
  intensity: 80
  units: kg
----------------------------------------------------------------
Exercise information
  Day 1
   Squats   5 x 5 @ 80kg
----------------------------------------------------------------
Program
 Week 1
  Day 1
   Squats   5 x 5 @ 80kg

 Week 2
  Day 1
   Squats   5 x 5 @ 80kg

 Week 3
  Day 1
   Squats   5 x 5 @ 80kg

 Week 4
  Day 1
   Squats   5 x 5 @ 80kg

----------------------------------------------------------------


## Using dynamic exercises

In [2]:
from streprogen import StaticExercise, DynamicExercise, Day, Program

# Create a 8 week training program
program = Program("Simple program", duration=8)


with program.Day() 

# Create a dynamic exercise and a static exercise
squats = DynamicExercise(
    "Squats", start_weight=100, final_weight=110, min_reps=3, max_reps=8
)
biceps = StaticExercise("Biceps", "3 x 12")
day = Day(exercises=[squats, biceps])

# Add the day to the program and render it
program.add_days(day)
program.render()

# Save the program as a HTML file
with open("simple_program.html", "w", encoding="utf-8") as file:
    # Control table width by passing the 'table_width' argument
    file.write(program.to_html(table_width=8))

The output file generated by the code above is:

* [simple_program.html](simple_program.html)

## Several days

This example introduces several new features:
    
* Controlling the sum of the repetitions for an exercise with `reps_per_exercise`.
* Controlling the average intensity (% of maximum weight) using `intensity`.
* Controlling the rounding globally with `round_to`.

In [3]:
from streprogen import StaticExercise, DynamicExercise, Day, Program

# Create a 6 week training program with 20 reps per exercise
program = Program(
    "Program with several days",
    duration=8,
    reps_per_exercise=20,
    intensity=70,
    round_to=5,
)

# Common keyword arguments for all dynamic exercises
kwargs = {"min_reps": 4, "max_reps": 8}

# Create the first day
squats = DynamicExercise("Squats", 100, 120, **kwargs)
bench = DynamicExercise("Bench press", 80, 95, **kwargs)
dayA = Day("Day A", exercises=[squats, bench])

# Create the second day
squats = DynamicExercise("Squats", 100, 110, **kwargs)
deadlifts = DynamicExercise("Deadlifts", 120, 135, reps=15, **kwargs)
# The number of reps for deadlifts overrides the global
# value of 20 and is instead set to 15.
dayB = Day("Day B", exercises=[squats, bench])

# Add the days to the program and render it
program.add_days(dayA, dayB)
program.render()

# Save a .html file
with open("prog_several_days.html", "w", encoding="utf-8") as file:
    file.write(program.to_html(table_width=8))

/home/tommy/Desktop/github/streprogen/streprogen/program.py:352: UserWarning: 
This maps to intensities in the range [72.0, 86.0], but the goal average intensity is 70,
which is not achievable with this rep range.
SOLUTION: Either (1) change the repetition range, (2) change the desired intensity
or (3) ignore this message. The software will do it's best to remedy this.

  warnings.warn(msg)
/home/tommy/Desktop/github/streprogen/streprogen/program.py:352: UserWarning: 
This maps to intensities in the range [72.0, 86.0], but the goal average intensity is 70,
which is not achievable with this rep range.
SOLUTION: Either (1) change the repetition range, (2) change the desired intensity
or (3) ignore this message. The software will do it's best to remedy this.

  warnings.warn(msg)
/home/tommy/Desktop/github/streprogen/streprogen/program.py:352: UserWarning: 
This maps to intensities in the range [72.0, 86.0], but the goal average intensity is 68,
which is not achievable with this rep range

The output file generated by the code above is:

* [prog_several_days.html](prog_several_days.html)

## A realistic program

In [4]:
from streprogen import StaticExercise, DynamicExercise, Day, Program
import subprocess  # Used to run pdflatex

# Create a function to map from start weights to final weights
# it adds 1.5% of the starting weight every week
def f(start_weight):
    percentage_growth = 0.015
    percentage = start_weight * percentage_growth
    final_weight = start_weight + percentage * duration
    return int(round(final_weight))


# Create a function for the static exercise
def dips_scheme(week):
    if week <= 4:
        return "4 x 10 @ bodyweight"
    else:
        return "4 x 12 @ bodyweight + 10kg"


# Create the program
duration = 8
program = Program("A realistic program", units="", round_to=2.5, duration=duration)

# The first day
day1 = Day("Monday _1")
squats = DynamicExercise("Squats", 95, f(95))
chins = DynamicExercise("Chins (light)", 100, f(100))
press = DynamicExercise("Military press", 50, f(50))
day1.add_exercises(squats, chins, press)

# The second day
day2 = Day("Wednesday")
deadlifts = DynamicExercise("Deadlifts", 120, f(120))
bench_press = DynamicExercise("Bench", 70, f(70))
chin_ups = DynamicExercise("Chin ups", 100, f(100))
# Notice that 'dips_scheme' is a function
dips = StaticExercise("Dips", dips_scheme)
day2.add_exercises(deadlifts, bench_press, chin_ups, dips)

# The third day
day3 = Day("Friday")
squats = DynamicExercise("Squats", 85, f(85))
bench = DynamicExercise("Bench (light)", 85, f(85))
rows = DynamicExercise("Rows", 65, f(85))
day3.add_exercises(squats, chins, press)

# Add the days and render the program
program.add_days(day1, day2, day3)
program.render()


# Save a .html file
with open("realistic_program.html", "w", encoding="utf-8") as file:
    file.write(program.to_html(table_width=6))

# Save a .tex file
with open("realistic_program.tex", "w", encoding="utf-8") as file:
    # Clear pages will put one week per page in the document
    file.write(program.to_tex(table_width=8, clear_pages=True))

# Use pdflatex to create a .pdf from the .tex file
ret = subprocess.call(["pdflatex", "realistic_program.tex"], shell=False)

The output file generated by the code above is:

* [realistic_program.html](realistic_program.html)
* [realistic_program.tex](realistic_program.tex)
* [realistic_program.pdf](realistic_program.pdf)